<a href="https://colab.research.google.com/github/FunixSE00292x/Funix_BGD301x_Dataset/blob/main/BGD301x_Assignment_01_ducnase00292x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup - Assignment 1 BigData - BGD301x - ducnase00292x@funix.edu.vn

## Dowload Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


## Import thư viện

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ff1c86cb4d2c7d2fccd3c78fdfc6c4cb8bea5cfec08420b6e0107273b0b991d1
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from operator import add
from re import sub

## Đọc dữ liệu raw -> convert sang list\<int\> (RDD)

In [ ]:
rdd = (
  SparkContext.getOrCreate()
    .textFile("/content/drive/MyDrive/Colab Notebooks/assn1/*.txt") # đọc từ file
    .map(lambda raw_data: sub("[\D]", " ", raw_data)) # loại bỏ dấu câu
    .flatMap(lambda x: x.split()) # split văn bản thành list các số và hợp nhất 5 cái list này của 5 file lại thành 1 list chung
    .map(int) # convert list<string> -> list<int>
)

# Bài làm:

## Câu 1: Tìm số nguyên lớn nhất.

In [ ]:
def max_in_partition(iterator):
  yield max(iterator)

In [ ]:
print(
  rdd.repartition(10)  # tách thành 10 phần khác nhau
    .mapPartitions(max_in_partition)  # tìm max của mỗi phần riêng biệt - reduce lần 1
    .reduce(max) # tìm max giữa 10 giá trị max trả về của 10 phần -  reduce lần 2
)

4339988962


## Câu 2: Tính trung bình của tất cả các số nguyên.

In [ ]:
print(rdd.mean())

499494337.4409073


## Câu 3. Tìm mười số nguyên có tần suất xuất hiện cao nhất sử dụng hàm có sẵn trong pyspark.

In [ ]:
print(
  rdd.map(lambda num: (num, 1)).reduceByKey(add)
    .takeOrdered(10, key=lambda num: -num[1]) # Sort by values - descending
    # .sortBy(lambda num: num[1], ascending=False).take(10)
)

[(339988962, 8), (830500257, 5), (793327022, 5), (983233161, 3), (268141338, 3), (781491805, 2), (494040970, 2), (955884525, 2), (645970095, 2), (212849680, 2)]


## Câu 4. Tìm mười số nguyên có tần suất xuất hiện cao nhất bằng cách tự viết thuật toán MergeSort.

In [ ]:
def custom_merge_sort(data, key):
    def merge_sort(arr):
        size = len(arr)
        if size > 1:
            middle = size // 2
            left_arr = arr[:middle]
            right_arr = arr[middle:]

            merge_sort(left_arr)
            merge_sort(right_arr)

            x = 0
            y = 0
            z = 0

            left_size = len(left_arr)
            right_size = len(right_arr)
            while x < left_size and y < right_size:
                if key(left_arr[x]) < key(right_arr[y]):
                    arr[z] = left_arr[x]
                    x += 1
                else:
                    arr[z] = right_arr[y]
                    y += 1

                z += 1

            while x < left_size:
                arr[z] = left_arr[x]
                x += 1
                z += 1

            while y < right_size:
                arr[z] = right_arr[y]
                y += 1
                z += 1
    merge_sort(data)
    return data

In [ ]:
list_integers = rdd.map(lambda num: (num, 1)).reduceByKey(add).collect()
sorted = custom_merge_sort(list_integers, lambda x: -x[1])
print(sorted[:10])

[(339988962, 8), (793327022, 5), (830500257, 5), (268141338, 3), (983233161, 3), (310748479, 2), (983189434, 2), (401610809, 2), (284860464, 2), (147137089, 2)]
